In [ ]:
# Importar as bibliotecas

import datetime
import json
import os
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Trazer as opções de configuração do JSON

json_file = os.path.abspath('../options.json')

with open(json_file, 'r', encoding='utf-8') as j_file:
    json_data = json.load(j_file)

In [ ]:
# Chamada dos arquivos

archive_dir = os.path.abspath('../data')

cvm_dir = os.path.join(archive_dir, 'cvm')
cdi_dir = os.path.join(archive_dir, 'cdi')

cvm_dir_processed = os.path.join(cvm_dir, json_data['DIR']['CVM']['DATA_NAME'])

In [ ]:
# Gráfico de evolução do preço das cotas

cvm_files = os.listdir(cvm_dir_processed)
all_data = []

DATE_START = json_data['CONFIG']['START_DATE']
DATE_END = json_data.get("CONFIG", False).get("END_DATE", False) or str(datetime.date.today())

for investment_file in cvm_files:
    investment_path = os.path.join(cvm_dir_processed, investment_file)
    investment = pandas.read_csv(investment_path, sep=';', encoding='utf-8')

    investment = investment.query('@DATE_START <= DT_COMPTC <= @DATE_END')

    investment = investment[['DT_COMPTC', 'VL_QUOTA', 'NOME_FUNDOS', 'CNPJ_FUNDO_CLASSE']].copy()
    all_data.append(investment)

investments = pandas.concat(all_data, ignore_index=True)
investments['DT_COMPTC'] = pandas.to_datetime(investments['DT_COMPTC'])
investments = investments.sort_values('DT_COMPTC')

In [ ]:
# Gráfico de evolução do preço das cotas

sns.set_style("whitegrid")
plt.figure(figsize=(20, 6))

plot = sns.lineplot(
    data=investments, 
    x='DT_COMPTC', 
    y='VL_QUOTA', 
    hue='NOME_FUNDOS', 
    palette=['magenta', 'deepskyblue', 'yellowgreen'],
    linewidth=2.5
)

plt.title("Evolução do Valor da Cota", fontsize=16, fontweight='bold', pad=20)
plt.text(
    1.225, 0.0,
    "Atualizado em " + str(datetime.datetime.today().date()), 
    transform=plt.gca().transAxes, 
    fontsize=10, 
    va='bottom', ha='left'
)

plt.xlabel("Período", fontsize=14)
plt.ylabel("Valor da Cota (R$)", fontsize=14)
plt.xticks(
    rotation=45, 
    fontsize=11,
    ha='right',
    rotation_mode= 'anchor'
)
plt.legend(
    title='Fundo de Investimento', 
    bbox_to_anchor=(1.05, 1), 
    fontsize=11
)

plt.gca().tick_params(axis='x', pad=5)
plt.show()


In [ ]:
# Gráfico de valorização

def valuation_comparable():
    cvm_list = json_data['GRAPHIC']['INVESTMENT_LIST']
    metric_list = json_data['GRAPHIC']['METRICS_LIST']

    CSV_EXTENSION = '.csv'

    cvm_archives = []
    metrics_archives = []

    for cvm_cnpj in cvm_list:
        cvm_name = json_data['INVESTMENTS'][cvm_cnpj]['Fundo']
        cvm_name = cvm_name.replace(' ', '_')

        cvm_archive = os.path.join(cvm_dir, json_data['DIR']['CVM']['DATA_NAME'], cvm_name + CSV_EXTENSION)
        cvm_archive = pandas.read_csv(cvm_archive, index_col=False, sep=';')
        
        cvm_archives.append(cvm_archive)

    for metric_key_name in metric_list:
        metric_archive = os.path.join(archive_dir, metric_key_name.lower(), metric_key_name.lower() + '_valuation' + CSV_EXTENSION)
        metric_archive = pandas.read_csv(metric_archive, index_col=False, sep=';')

        metrics_archives.append(metric_archive)
    
    return [ cvm_archives, metrics_archives ]


In [ ]:
df_cvm_archives, df_metrics_archives = valuation_comparable()

sns.set_style("whitegrid")
plt.figure(figsize=(20, 6))

plot = sns.lineplot(
    data=investments, 
    x='DT_COMPTC', 
    y='VL_QUOTA', 
    hue='NOME_FUNDOS', 
    palette=['magenta', 'deepskyblue', 'yellowgreen'],
    linewidth=2.5
)

plt.title("Evolução do Valor da Cota", fontsize=16, fontweight='bold', pad=20)
plt.text(
    1.225, 0.0,
    "Atualizado em " + str(datetime.datetime.today().date()), 
    transform=plt.gca().transAxes, 
    fontsize=10, 
    va='bottom', ha='left'
)

plt.xlabel("Período", fontsize=14)
plt.ylabel("Valor da Cota (R$)", fontsize=14)
plt.xticks(
    rotation=45, 
    fontsize=11,
    ha='right',
    rotation_mode= 'anchor'
)
plt.legend(
    title='Fundo de Investimento', 
    bbox_to_anchor=(1.05, 1), 
    fontsize=11
)

plt.gca().tick_params(axis='x', pad=5)
plt.show()


print(df_cvm_archives)